---
title: "Writing Stable Diffusion from Scratch 5"
reading-time: 
date: "2023-3-23"
categories: [fastaipart2,Stable-Diffusion]
---

After This you should be comfortable with following : <br>
1- Chain Rule <br>
2- Basic linear NN architecture <br>
3- SGD concept <br>
4- Coding the model in python and pytorch <br>

So now we're going to take a look at notebook number three in the normal repo course22p2, and we're going to be looking at the forward and backward passes of a simple multi-layer perceptron, a neural network. The initial stuff up here is just importing things and just settings and stuff that they're just copying and pasting some stuff from previous notebooks around paths and parameters and stuff like that. So we'll skip over this now. So will often be kind of copying and pasting stuff from one notebook to another. That's kind first cell get things set up and have also loading in our data for MNIST as tensors. Okay, so we to start with, need to create the basic architecture of our neural network. And I did mention at the start of the course that we will briefly review everything that we need to cover. So we should briefly review what basic neural networks are and why they are what they are. So to start with, let's consider a linear model. Oops, that's not how I do it,let's take the most simple example possible, which is we're going to pick a single pixel from from our MNIST pictures. And so that will be our x and for our, for our y values. Then we'll have some lost function of how good is this model? Sorry, not some lost function that's even simpler for our y value. We're going to be looking at how likely is it that this is, say, the number three based on the value of this one pixel? So the pixel, its value will be x and the probability of being the number three we'll call y and if we just have a linear model, then it's going to look like this. And so in this case, it's it's saying that the brighter this pixel is, the more likely it is that it's the number three. And so there's a few problems with this. The first one, obviously, is that as a linear model, it's very limiting because maybe, you know, we actually are trying to draw something that looks more like this. So how would you do that? Well, there's actually a neat trick we can use to do that. What we could do is, well, let's first of all, talk about something we can't do, something we can't do is to add a bunch of additional lines. So consider what happens if we say, okay, well, let's add a few different lines. 

So let's also add this line. So what would be the sum of our two lines? Well, the answer is, of course, that the sum of the two lines will itself be a line. So it's not going to help us at all match the actual curve that we want. So here's the trick. Instead, we could create a line like this that actually we could get this line. And now consider what happens if we add this original line with this new what's not a line, right? It's it's two line segments. So what we would get is this everything to the left of this point is going to not be changed. If I add these two lines together, because this is zero all the way and everything to the right of it is going to be reduced. It looks like they've got similar slopes, so we might end up with instead. So this would all disappear here and instead we would end up with something like this. And then we could do that again, right? We could add an additional line that looks a bit like that. So it would go, but this time it could go even further out here and it could be something like this. Say, so what if we added that? Well, again, at the point underneath here, it's always zero, so it won't do anything at all. But after that it's going to make it even more negatively sloped. And if you can see using this approach, we could add up lots of these rectified lines. These lines are truncated zero, and we could create any shape we want with enough of them. And these lines are very easy to create because actually all we need to do is to create just a regular line, just to get a regular line. Right. Which we can move up, down, left, right, change its angle, whatever, and then just say if it's greater than zero, truncate it to zero, or we could do the opposite because through a line going the opposite direction, it has less than zero, we could say truncated to zero, and that would get rid of as we want. This whole section here and make it flat. Okay. So these are rectified lines and so we can sum up a bunch of these together to basically match any arbitrary curve.

(Bahman note )This is amazing , the is intuition of a theory that we can create or mimic any funtion with neural net. Like creating anything fron linear model. (I do not remember the paper or the creator / he is very famous , he said something like this in his paper but did not try for more layers and years after that when people did that new era of AI started) . (Bahman note )

 So let's start by doing that. Oh, the other thing we should mention, of course, is that we're going to have not just one pixel, but we're going to have lots of pixels. So to start with, the kind of most, you know, slightly, you know, the only slightly less simple approach, we could have something where we've got, you know, pixel number one and pixel number two, we're looking at two different pixels to see how likely they are to be the number three. And so that would allow us to draw more complex shapes that have some kind of surface between them. Okay. And then we can do exactly the same thing is to create these surfaces. 

We can add up lots of these rectified lines together, but now they're going to be kind of rectified planes, but it's going to be exactly the same thing. We're going to be adding together a bunch of lines, each one of which is truncated at zero. Okay. So that's the quick review. And so to do that, we'll start out by just defining a few variables. So n is the number of training examples. m Is the number of pixels. c is the number of possible values of our digits. And so here they are, (50000, 784, tensor(10)). Okay. So what we do is to is we basically decide ahead of time how many of these lines segment thing is to add up. And so the number that we create in a layer is called the number of hidden nodes or activations. So we call that nh, So let's just arbitrarily decide on creating 50 of those. So in order to create lots of lines which where they're going to truncate at zero, we can do a matrix multiplication. So with the matrix multiplication, we're going to have something where we've got by 500000 rows by 784 column. Yeah, by 784 columns. And we're going to multiply that by something with 784 rows and ten columns and why is that? Well, that's because if we take this very first line of this first vector here, write one of 784 values there. The pixel values of the first image. Okay, So this is our first image. And so they're each going to each of those 700 different values, but we multiply it by each of these 784 values and in the first column, the zero index column. And that's going to give us a number in our output. So our output is going to be And so that result will multiply those together and add them up and that result is going to end up over here in this first. So and so each of these columns is going to eventually represent if this if this was a this is a linear model in this case, this is just the example of doing a linear model, each of these cells is going to represent the probability. So this first column will be the probability of being zero, and the second column will be the probability of one. The third column will be the probability of being a two and so forth. So that's why we're going to have these ten columns, each one allowing us to write the 784 inputs. Now of course, we're going to do something bit more tricky than that, which is actually we're going to have a 74 by 50 input going into a 784 by 50 output to create the 50 hidden layers. Then we're going to truncate those at zero and then multiply that by a 50 by 10 to create an output. So we do it in two steps. So and the way it sgd works is we start with just this is our weight matrix here and this is our data, this is our outputs. The way it works is that this weight matrix is initially filled with random values. So called this contains our pixel values. This contains the results. So w is going to start with random values. So here's our weight matrix. It's going to have, as we discussed, 50,000 by 50 random values and it's not enough just to multiply. We also have to add. So that's what makes it a linear function. So we call those the biases, the things we add. We can just start those zeros, so we'll need one for each output. So 50 of those and so that'll be layer one. And then as we just mentioned, layer two will be a matrix that goes from 50 hidden. And now I'm going to do something totally cheating. To simplify some of the calculations for the calculus, I'm only going to create one output.



## The forward and backward passes

In [1]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

In [2]:
from urllib.request import urlretrieve
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [3]:
!ls -l data

total 16656
-rw-r--r-- 1 root root 17051982 Mar 27 04:43 mnist.pkl.gz


!ls -l data

In [4]:
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## Foundations version

### Basic architecture

In [5]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [6]:
# num hidden
nh = 50

 Well, I'm going to create one output. That's because I'm not going to use cross entropy just yet. Instead, I'm going to use MSE. So actually I'm going to create one output, which will literally just be what do i think it is? from 0 to 10. And so then we're going to compare those to the actual. So these will be our y predicted to only use a little hat for that and we're going to compare that to our actuals. And yeah, in this very hacky approach, let's say we predict over here the number nine and the actual is the number two and we'll compare those together using MSE, which will be a stupid way to do it because it's saying that nine is further away from being true than two, nine is farther away from true than it is from four in terms of how correct it is, which is which is not what we want at all. But this is what we're going to do just to simplify our starting point. So that's why we're going to have a single output for this weight matrix in a single output for this bias. So linear, let's create a function for putting extra linear layer with these weights in these biases. So it's a matrix multiply and an add. All right. So we can now try it. So if we multiply we doing x_valid this time? So just to clarify x_valid is 10,000 by 784. So if we put x_valid through our weights and biases with a linear layer, we end up with a 10,000 by 50. So 10,050 long hidden activations, they're not quite ready yet because we have to put them through relu. And so we're going to clamp at zero, so anything under zero will become zero. And so here's what it looks like when we go through the linear layer and then the relu here. And you can see he has a tensor with a bunch of things, some of which are zero, or they're positive. And so that's the result of this match, this matrix multiplication. Okay. So to create a basic MLP multi-layer perceptron from scratch, we will take our mini batch of X's x, b is an x match. We will create our first layers output with a linear and then we will put that through over here and then that will go through the second linear. So the first one uses the w1, b1. Okay, these ones and the second one uses the w2 to b2. And so we've now got a simple model and as we hoped, when we pass in the validation set, we get back 10000 digits. So that's a good start. Okay, so let's use our ridiculous lost function of MSE. So our results is 10,000 by one and our y_valid is just a vector. Now what's going to happen if I do res minus y-valid? 

In [7]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [8]:
def lin(x, w, b): return x@w + b

In [9]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [10]:
def relu(x): return x.clamp_min(0.)

In [11]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [12]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [13]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

### Loss function: MSE

So before you continue in the video, have a think about that. What's going to happen if I do res minus y_valid by thinking about the numpy broadcasting rules we've lent? Okay, let's try it. Oh terrible. We've ended up with a 10,000 by 10,000 matrix. So 100 million points now we would expect an MSA to contain a thousand points. Why does that happen? The reason it happened is because we have to start out at the last dimension and go right to left and we compare to the 10,000 to the one and say, are they compatible? And the answer is, that's right. Alexi And the chats got it right, broadcasting roles. So the answer is that this one will be broadcast over these 10,000. So this pair here will give us the next one and we'll also move here to the next one. Oh, there is no next one. What happens now? If you remember the rules in search, say, unit access for us. So I now have 10,000 by one. So that means each of the 10,000 outputs from here is will end up being broadcast across the 10,000 rows here. So that means that will end up for each of those 10,000. We'll have another 10,000. So we'll end up with a 10,000 by 10,000 output. So that's not what we want. So how could we fix that? Well, what we really would want, would we want this to be If that was 10,000 comma one, then we'd compare these two right to left and they're both one. So those those match and there's nothing to broadcast because they're the same. And then we'll go to the next one, 10000 to 10000 those match. So that just go element wise for those and we'd end up with exactly what we want it to hand out with 10,000 results. Or alternatively we could remove this dimension and then again, same thing we're going to add right to left compatible 10,000. So they will get element wise operation. So in this case I got rid of the trailing comma one. It's a couple of ways you could do that. One is just to say, okay, grep every row and the zeroth column of res and that's going to turn it from a 10,000 by one into a 10,000. 

Or alternatively we can say dot squeeze. Now dot squeeze removes all trailing unit vectors and possibly also prefix unit vectors. I can't quite recall. I guess we should try. So let's say res None comma calling, comma None,  q.shape. Okay. So if I go q.squeeze. shape. Okay. So all the unit vectors get removed. Sorry, all the unit dimensions get removed I should say. Okay. So now that we've got a way to remove that axis that we didn't want, we can use it. And if we do that subtraction now we get 10,000 just like we wanted it. So now let's get our training and validation wise, we'll turn them into floats because we're using MSI. So let's calculate our predictions for the training set, which is 50,000 by one. And so if we create an MSE function that just does what we just said we wanted, so it does a subtraction and then squares it and then takes the mean that the MSE. So there we go. We now have a loss function being applied to our training set. Okay, now we need gradients. So as we briefly discussed last time, gradients slopes and in fact maybe it would even be easier to look at last time. So this is last times notebook. And so we saw how the gradient at this point is the slope here and so it's the as we discussed rise over run now so that means as we increase in this case time by one the distance increases by how much that's what the slope is. So why is this interesting? The reason it's interesting is because let's consider our neural network. Our neural network is some function that takes two things two groups of things. It contains a matrix of our inputs, and it contains our weight matrix. And we want to and let's assume we're also putting it through a loss function. So let's say, well, I guess we can be explicit about that. So we could say we then take the result of that and we put it through some lost function. 

So these are the predictions and we compare it to our actual dependent variable. So that's our neural net and that's our last function. Okay. So if we can get the derivative of the loss with respect to, let's say, one particular weight, so let's say weight, and I'm like number zero, what is that doing? Well, it's saying as I increase, increase the weight by a little bit, what happens to the loss? And if it says, oh, well, that would make the loss go down, then obviously I want to increase the weight by a little bit. And if it says, Oh, it makes the loss go up, then obviously I want to do the opposite. So that derivative of the loss with respect to the weights each one of those tells us how to change the weights. And so to remind you, we then change each weight by that derivative times a little bit and subtract it from the original weights. And we do that a bunch of times, and that's called SGD. Now, there's something interesting going on here, which is that in this case there's a single input and a single output, and so the derivative is a single number at any point. It's the speed. In this case, the vehicle is going. But consider a more complex function like, say, this one. Now, in this case, there's one output, but there's two inputs. And so if we want to take the derivative of this function, then we actually need to say, well, what happens if we increase x by a little bit? And also what happens if we increase y by a little bit? And in each case what happens to x? And so in that case, the derivative is actually going to contain two numbers, right? It's going to contain the derivative of x with respect to y and it's going to contain the derivative of z with respect to x. What happens if we change each of these two numbers? So, for example, these could be, as we discussed, two different weights in our neural network and Z could be our loss. For example, now we've got actually 784 inputs, right? So we would actually have 784 of these. So we don't normally write them all like that. We would just say, yes, this little squiggly symbol to say the derivative of the loss across all of them with respect to all of the weights. Okay. And that's just saying that there's a whole bunch of them. It's a shorthand way of writing this. Okay. So it gets more complicated still, though, because think about what happens if, for example, you were in the first layer where we've got a weight matrix that's going to end up giving us 50 outputs, right? So for every image, we're going to have 784 inputs to our function and we're going to have 50 outputs to our function.

And so in that case, I can't even draw it right because like for every even if I had two inputs and outputs, then as I increase my first input, I'd actually need to say, how does that change both of the two outputs? And as I change my second input, how does that change both of my two outputs? So for the full thing, you're actually going to end up with a matrix of derivatives. It basically says for every input that you change by a little bit, how much does it change every output of that function? So you're going to end up with a matrix. So that's what we're going to be doing, is we're going to be calculating these derivatives, but rather than being single numbers, they're going to actually contain matrices with a row for every input and a column for every output and a single cell in that matrix. Well, tell us, as I change this input by a little bit, how does it change this output Now? Eventually we will end up with a single number for every input, and that's because our loss in the end is going to be a single number. And this is like a a requirement that you'll find when you try to use SGD is that your loss has to be a single number. And so we generally get it by either doing the the sum or a mean or something like that. But as you'll see on the way there, we're going to have to be dealing with these matrix of derivatives. So I just want to mention, as I might have said before, I can't even remember there is this paper that Terence Power and I wrote a while ago, which goes through all this, and it basically assumes that you only know high school calculus. And if you don't checkout Khan Academy, but then it describes matrix calculus in those terms. So it's going to explain to you. Exactly. And it works through lots and lots of examples. So, for example, as it mentions here, when you have this matrix of derivatives, we call that a Jacobean matrix. So like there's all these like words, it doesn't matter too much if you know them or not, but it's convenient to be able to talk about, you know, the matrix of all of the derivatives. If somebody just says this Jacobean, it's a little convenient. It's a bit, a little bit easier than saying the matrix of all of the derivatives where all of the rows are things that are, are all the inputs and all the columns of the outputs. So yeah, if you want to really understand, get to a point where papers are easier to read particular, it's quite useful to know this notation and, and, and definitions of words.
https://explained.ai/matrix-calculus/

You can certainly get away without it. It's just something to consider. Okay, so we need to be able to calculate derivatives at least of a single variable. And I am not going to worry too much about that because that is something you do in high school math and b, because your computer can do it for you and so you can do it symbolically using something called sympy, which is really great. So if you create two symbols called x and y, you can say please differentiate x squared with respect to x. And if you do that sympy will tell you the answer is 2x. If you say differentiate three x squared plus nine with respect to x, sympy will tell you that six x and a lot of you probably will have used Wolfram Alpha that does something very similar. I kind of quite like this because I can quickly do it inside my notebook and included in my prose. So I think sympy is pretty cool. So you know, basically yeah. That if you, you know, you can quickly calculate derivatives on a computer. Having said that, I do want to talk about why the derivative of three x squared plus nine equals six x, because that's going to be very important. So three x squared plus nine. So we're going to start with the the information that that derivative of a to the b with respect to a equals b*a , So for example, the derivative of x squared with respect to x equals 2x. So that's just something I'm hoping you remember from high school or refresh your memory is in Khan Academy also. So there that is there. So what we could now do is we could rewrite this derivative as 3u plus nine and then we'll write u equals x squared. Okay, Now this is getting easier. The derivative of two things being added together is simply the sum of the derivatives. Oh, I forgot. B minus one of the exponent. Thank you. So B eight the power of b minus one. That's what it should be, which would be two x to the power of one and the one is not needed. Thank you for fixing that. All right. So we just some of them up so we get the derivative of 3u is actually just well, it's going to be the derivative of that plus the derivative of that. Now the derivative of any constant with respect to a variable is zero. Because if I change something an input, it doesn't change the constant, it's always nine. So that's going to end up as zero. And so we're going to end up with the dy/du equals something plus zero. And the derivative of 3u with respect to u is just three, because it's just a line that's its slope. Okay, But that's not dy/dx  Well, the cool thing is that dy/dx is actually just equal to dy/du * du/dx. So I'll explain why in a moment. But for now then let's recognize we've got du/dx = 2x, we know that one 2x so we can now multiply these two bits together and we will end up with two X times three, which is six x, which is what sympy they told us. So fantastic. Okay, this is something we need to know really well, and it's called the chain rule and it's best to understand it intuitively.

So to understand it intuitively, we're going to take a look at an interactive animation. So I found this nice interactive animation on this page here
https://webspace.ship.edu/msrenault/geogebracalculus/derivative_intuitive_chain_rule.html

And the idea here that we've got a wheel spinning around and each time it spins round, this is x going up. Okay. So at the moment there's some change in x,dx over a period of time. All right. Now this wheel is eight times bigger than this wheel. So each time this goes round, once, if we connect the two together, this wheel would be going round four times faster because the difference between the multiple between eight and two is four. Maybe I'll bring this up to here. So now that the this wheel is has got twice as big a circumference as the other wheel, each time this goes around, once this is going around two times. So the change in u each time X goes round, once the change in u will be two. So that's what du/dx is saying. The change in u for each change in x is two. Now we could make this interesting by connecting this wheel to this well. Now this wheel is twice as small as this wheel. So now we can see that again. Each time this spins round, once this spins round twice because this has twice the circumference of this. So therefore  dy/du equals two. But now that means every time this goes round, once this goes round twice every time this one goes round, once this one goes round twice. So therefore every time this one goes round once this one goes round four times So to dy/dx equals four. So you can see here how the two well how the, du/dx has to be multiplied with the dy/du to you to get the total. So this is what's going on in the chain role and this is what you want to be thinking about is this idea that you've got one function that is kind of this intermediary. And so you have to multiply the two impacts to get the impact of the X. Wheel, on the y Wheel, we'll help you find that useful. I find this personally, I find this intuition quite useful. So why do we care about this? Well, the reason we care about this is because we want to calculate the gradient of our MSE applied to our model. And so our inputs are going through a linear, they're going through a relu here, they're going through another linear, and then they're going through an MSE. So there's four different steps going on. And so we're going to have to combine those all together. And so we can do that with a chain rule. So if our steps are that lost function is the so we've got the lsot function, which is some function of the predictions and the actuals. And then we've got we've got the second layer is a function of actually let's say let's call this the output of the second layer slightly. We have notation, but hopefully it's not too bad. It's going to be a function of the relu of the activations and relu activation  are a function of the first layer and the first layer is a function of the inputs. Oh, and of course this also has weights and biases. So we're basically going to have to calculate the derivative of that. Okay. But then remember that this is itself a function. So then we need to multiply that derivative by the derivative of that, but that's also a function. So we have to multiply that derivative by this, but that's also a function. So we have to multiply that derivative by this. So that's going to be our approach. We're going to start at the end, we're going to take its derivative and then we're going to gradually keep multiplying as we go each step through. And this is called back propagation. So back propagation sounds pretty fancy, but it's actually just using the chain rule. Gosh, I didn't spell that very well.it's just using the chain rule. And as you'll see, it's also just taking advantage of a computational trick of memorizing some things on the way. And in our chat, Stephen made a very good point about understanding nonlinear functions in this case, which is just a consider that the wheels could be growing and shrinking all the time as they're moving, but you're still going to have the same compound effect, which I really like that.
Thank you, Sylvar There's also a question in the chat about why is this column comma zero being placed in the function given that we can do it outside the function? Well, the point is we want an MSE function that will apply to any output, not that we're not using it once we want it to work any time. So we haven't actually modified preds or anything like that or y_train. So we want this to be able apply to anything without us having to like preprocessor. It is basically the idea here. Okay, so let's take a look at the basic idea. So he is going to do a forward pass in a backward pass. So the forward passes where we calculate the loss. 


So the loss is, oh, I've got an error here that should be diff. Yeah, we go. So the loss is going to be the output of our neural net minus our target squared. Then take the mean, Okay. And then our output is going to be the output of the second linear layer. The second linear layer's input will be the relu, the relu
 input will be the first layer. 
 
 
It's going to take our input, put it through a linear layer, put that through a relu, put that through a linear layer and calculate the MSE. Okay, That bit hopefully is pretty straightforward. So 

what about the backward pass? So the backward pass, what I'm going to do and you'll see why in a moment is I'm going to store the gradients of each layer. So for example, the gradients of the loss with respect to its inputs in the layer itself. So I'm going to create a new attribute, I could call it anything I like, and it's going to call it (.g) .So I've got to create, a new attribute called out.g, which is going to contain the gradients. You don't have to do it this way, but as you'll see, it turns out pretty convenient. So that's just going to be two times the difference because we've got different squared, right? So that's just the derivative. And then we have taken the mean here. So we have to do the same thing here divided by the input shape. And so that's, that's those gradients, that's good. And now what we need to do is multiply by the gradients of the previous layer. So what are the gradients of a linear layer? I've created a function for that here. So the gradient is a linear layer. We're going to need to know the weights of the layer. We're going to need to know the biases of the layer. And then we're also going to know the input to the linear layer, because that's the thing that's actually being manipulated in here. And then we also going to need the output because we have to multiply by the gradients because we've got the chain role. So again, we're going to store the gradients of our input like inp.g . So this would be the gradients of our output with respect to the input (inp.g), and that's simply the weights because the weights. (w.t). So a matrix multiplier is just a whole bunch of linear functions. So each one slope is just its weight, but you have to multiply it by the gradient of the outputs because of the chain role and then the gradient of the outputs. With respect to the weights (w.g) is going to be the input times the output summed up. I'll talk more about that in a moment. The derivatives of the bias is very straightforward. It's the gradients of the output added together because the bias is just a constant value. So for the chain role, we simply just use output times one, which is output. So for this one here again, we have to do the same thing we've been doing before, which is multiply by the output gradients because of the chain rule and then we've got the input weights. So every single one of those has to be multiplied by the outputs. And so that's why we have to do an unsqueese minus one. So what I'm going to do now is I'm going to show you how I would experiment with this code in order to understand it, and I would encourage you to do the same thing. It's a little harder to do this one cell by cell because we kind of want to put it all into this function like this. So we need a way to explore the calculations interactively and the way we do that is by using the Python debugger.

Here is how you let me see a few ways to do this. Here's one way to use the Python debugger. The Python debugger is called PDB. So if you say PDB dot set trace in your code, then that tells the debugger to stop execution. When it reaches this line, it sets a breakpoint. So if I call forward and backward, you can see here it stopped and the interactive Python debugger I PDB has popped up with an arrow pointing at the line of code. It's about to run. And at this point there's a whole range of things we can do to find out what they are. We page for help understanding how to use the Python debugger. It's one of the most powerful things I think you can do to improve your coding. So one of the most useful things you can do is to print something. You see all these single letter things. They're just shortcuts. But in a debugger you want to be able to do things quickly instead of typing print, just type p. So for example, let's take a look at the shape of the input. So I type p for print input shape. So I've got a 50,000 by 50 input to the last layer that makes sense. These are the hidden activations coming in to the last layer for every one of our images. What about the output gradients? And there's that as well. And actually a little trick you can ignore that you don't have to use the p at all if your variable name is not the same as any of these commands.
So I could have just typed out.g.shape, get the same thing. Okay. So you can also put in expressions so let's have a look at the shape of this. So the output of this is let's see if it makes sense. We've got the input 50,000 by 50. We put a new axis on the end unsqueeze minus one is the same as doing dot is indexing it with dot dot dot comma None sets but a new axis at the end. So that would have become And then the out.g.unsqueeze. We're putting in the first dimension, so we're going to have 50,000 by 50 by one times 50,000 by one by one. And so we're going to end we're going to end up getting this broadcasting happening over these last two dimensions, which is why we end up with 50,000 by 50 by one. And then with summing up, this makes sense, right? We want to sum up over all of the inputs, each image is individually contributing to the derivative. And so we want to add them all up to find their total impact. Because remember the sum of a bunch of the derivative of the sum of functions is the sum of the derivatives of the functions. So we can just some of them up. Now, this is one of these situations where if you see a times and a sum and unsqueeze, it's not a bad idea to think about Einstein summation notation. Maybe there's a way to simplify this. So first of all, let's just see how we can do some more stuff in the debugger. I'm going to continue so just continue running. 

So press c for continue and it keeps running until it comes back again to the same spot. And the reason we've come to the same spot twice is because Lin_grad is called two times. So we would expect that the second time we're going to get a different bunch of inputs and outputs. And so I can print out a couple of the inputs and output gradient. So now, yes, So this is the first layer going into the second layer. So that's exactly what we expect to find out What called this function, you just type w is where am I? And so you can see here, where am I? Oh, forward and backward was called. See the arrow that called lin_grad the second time and now we're here in w.g equals. If we want to find out what actually ends up being equal to. I can press n for to say go to the next line and so now we've moved from line 5 to line 6,  the instruction point is now looking at line six. So I could now print out, for example, w.g.shape, and that's the shape our weights. One person on the chat has pointed out that you can use breakpoint instead of this import PDB business. Unfortunately, the breakpoint keyword doesn't currently work in Jupyter or in IPython, so we actually can't. Sadly. That's why I'm doing it the old fashioned way. So this way maybe they'll fix the bug at some point. But for now we have to type all this. Okay, so those are a few things to know about, but I would definitely suggest looking up a Python PDB tutorial to become very familiar with this incredibly powerful tool because it really is so very handy. So if I just press continue again, it keeps running all the way to the end and it's now finished running forward and backward. 
https://realpython.com/python-debugging-pdb/




(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [14]:
res.shape,y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [15]:
(res-y_valid).shape

torch.Size([10000, 10000])

We need to get rid of that trailing (,1), in order to use `mse`.

In [16]:
res[:,0].shape

torch.Size([10000])

In [17]:
res.squeeze().shape

torch.Size([10000])

In [18]:
(res[:,0]-y_valid).shape

torch.Size([10000])

In [19]:
y_train,y_valid = y_train.float(),y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [20]:
def mse(output, targ): return (output[:,0]-targ).pow(2).mean()

In [21]:
mse(preds, y_train)

tensor(4308.76)

### Gradients and backward pass

In [22]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [23]:
diff(3*x**2+9, x)

6*x

In [24]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [25]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

So when it's finished, we would find that there will now be, for example, w1.g, because this is this is the gradients that it just calculated and there would also be a  x_train.g and so forth. Okay, so let's see if we can simplify this a little bit. So I would be inclined to take these out and give them their own variable names just to make life a bit easier. It would have been better if I'd actually done this before the debugging, so it be a bit easier to type. So let's set i and o equal to input and output .g.unsqueeze. And actually let's get rid of the that we just call it. So i will be input. All right. So we've got here actually now let's put the end sequences back. I changed my mind. So. So just by. Oh, okay. So we'll get rid of our breakpoint and double check that we've got our gradients. Okay. And I guess before we re run out, we should probably set those to zero. So what I would do here to try things out is I put my breakpoint there and then I would try things. So let's go next. And so I realize here that what we're actually doing is we're basically doing exactly the same thing as einsum. Some would say. So I could test that out by trying an einsum. Right. Because I've just got this is being replicated. And then I'm summing over that dimension because that's the multiplication that I'm doing. So I'm basically multiplying the first dimension of and then summing over that dimension. So I could try running that and oh, it works. So that's interesting. I'd be easier if I just read this shaped bit long, isn't it,  dot shape. Okay, so it's a 50 by one. Oh, and I've got zeros because I did exchange that zero. That was silly. Let's try that again. That should be done. Gradient.zero. Okay, so let's try doing an einsum. And there we go. That seems to be working. That's pretty cool. So we're we've multiply at this repeating index. So we were just multiplying the first dimensions together and then summing over them. So there's no i here. Now, that's not quite the same thing as a matrix multiplication, but we could turn it into the same thing as more metaphor application just by swapping in i,and j so that the other way around that way would have  j, i comma , i k we can swap into dimensions very easily. That's what's called the transpose. So that would become a matrix multiplication if we just use the transpose. And in numpy the transpose is the capital T attribute. So here is exactly the same thing. Using a matrix multiply and a transpose. And let's check. Yeah, that's the same thing as well. Okay, cool. So that tells us that now we've checked in our debugger that we can actually replace all this with a matrix. Multiply. We don't need that anymore. Let's see if it works. That does. All right. x_trained.g. Okay. So hopefully that's convinced you that the debugger is a really handy thing for playing around with numeric programing ideas or coding in general. And so I think now is a good time to take a break. So let's take a eight minute break and I'll see you back here after a seven minute break. I'll see you back here in. 7 minutes. Thank you. 

All right. I'm back. What have I missed? Hmm? Okay. Welcome back. So we've calculated our derivatives, and we want to test them. Luckily, PyTorch already has derivatives implemented, so I've got a totally cheat. And. And is PyTorch to calculate the same derivatives. So don't worry about how this works yet, because I'm actually going to be doing all this from scratch anyway. For now, I'm just going to run it all through PyTorch and check that that derivatives are the same as ours. And they are. So we're on the right track. Okay. So this is all pretty clunky. I think we can all agree and obviously it's clunkier than what we do in PyTorch. 

In [26]:
forward_and_backward(x_train, y_train)

In [27]:
# Save for testing against later
def get_grad(x): return x.g.clone()
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

We cheat a little bit and use PyTorch autograd to check our results.

In [28]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad, chks))

In [29]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [30]:
loss = forward(xt2, y_train)
loss.backward()

In [31]:
for a,b in zip(grads, ptgrads): test_close(a, b.grad, eps=0.01)

So how do we simplify things? That's a really cool refactoring that we can do. So what we're going to do is we're going to create a whole class for each of our functions, for the regular function and for the linear function. So the the way that we're going to do this is we're going to create a dunder call. What is dunder call do let me show you. So if I create a class and we're just going to set that to print, hello. So if I create an instance of that class and then I call it as if it was a function oops, missing the dunder bit here, call it as if it's a function. It says hi. So in other words, you know, everything can be changed in Python. You can change how a class behaves, you can make it look like a function. And to do that you simply define dunder call your passing it an argument like so. Okay, so that's what dunder call does. It just says it's just a little bit of syntax, sugary kind of stuff to say. I want to be able to treat it as if it's a function without any method at all. You can still do it the method way you could have done this. I don't know why you'd want to, but you can. But because it's got this special magic named dunder call, you don't have to write that time to dunder call at all. So here, if we create an instance of the earlier class, we can treat it as a function. And what it's going to do is it's going to take its input and do the relu on it. But if you look back at the forward and backward, there's something very interesting about the backward pass, which is that it has to know about, for example, this intermediate gets passed over here, this intermediate calculation gets passed over here because of the chain role. We're going to need some of the intermediate calculations and not just because the chain rule, but because of actually how the derivatives are calculated. So we need to actually store each of the layer intermediate calculations. 

And so that's why value doesn't just calculate and return the output, but it's also stores its output and it also stores its input. So that way then when we call backward, we know how to calculate that we set the inputs gradient because remember we stored the input. So we can do that right and it's going to just be oh input greater than zero dot float. Right? So that's the definition. Okay. Of the derivative of a relu and then chain rule. So that's how we can calculate the forward pass and the backward pass for relu. And we're not going to have to then store all this intermediate stuff separately. It's going to happen automatically so we can do the same thing for a linear layer. Now linear layer needs some additional state weights and biases. relu doesn't, right? So there's no in it. So when you create a linear layer, we have to say what are its weights, what are its biases? We store them away. And then when we call it when the forward pass, just like before we store the input. So that's exactly the same line here. And just like before we calculate the output and store it and then return it. And this time of course we just call Lin and then for the backward pass, it's the same thing. Okay, so the input gradients we calculate just like before, oh .t brackets is exactly the same with a little t as big T is as a property. So that's the same thing. That's just the transpose. Calculate the gradients of the weights again with a chain rule and the bias, just like we do that before and they're all being stored in the appropriate places. And then for MSE, we can do the same thing. We don't just calculate the MSE, but we also store it and also now the MSE. And it just needs two things an input and a target. So we'll store those as well so that in the backward pass we can calculate its gradient of the input as being two times the difference. And there it all is. Okay, so our model now it's much easier to define. We can just create a bunch of layers, linear one,, w1,b1 relu, linear ,w2,b2, and then we can store an instance of the MSE. So this is not calling MSE, it's creating an instance of the MSE class, and this is an instance of the Lin class. 

This is an instance of relu class. So that is being stored. So then when we call the model, we pass it, our inputs in our target. We go through each layer, set x equal to the result of calling that layer and then pass that to the loss. So there's something kind of interesting here that you might have noticed, which is that Something interesting here is that we don't have two separate functions inside and inside our model, the lost function being applied to a separate neural net. But we've actually integrated the lost function directly into the neural net, into the model, see how the loss is being calculated inside the model. Now, that's neither better nor worse than having it separately. It's just different. And generally a lot of hugging faced stuff does it this way. They actually put the loss inside the forward Most stuff in FastAI and a lot of other libraries does it separately, which is the loss is a whole separate function and the model only returns the result of putting it through the layers. So for this model we're going to actually do the loss function inside model. So for backward, we just do each thing so self.loss.backward . So that self taught losses the MSE object. So that's going to call backward, right? And it's stored when it was called here it was storing remember the inputs, the targets, the outputs so it can calculate the backward and then we go through each layer is in reverse. Right. This is back propagation backwards, reversed, calling backward on each one. So that's pretty interesting. I think. So now we can calculate the model, we can calculate the loss we can call backward, and then we can check that each of the gradients that we stored earlier equal to each of our new gradients

Okay. So Williams asked a very good question that if you do puts put the loss inside here, how on earth do you actually get predictions? So generally what happens is in practice, huggingface models do something like that. So say self.preds equals x and then they'll say self.final_loss  = self.loss(x,targ) that and then return self.final_loss. And that way I guess you don't even need that last bit. Well anyway, this is what they do so I'll leave it there. And so that way you can kind of check like model.preds for example. So it'll be something like that. Or alternatively, you can return not just the loss, but both as a dictionary, stuff like that. So a few different ways you could do it actually. Now think about it. I think that's what they do is they actually return both as a dictionary. So it would be, it'd be like return dictionary loss equals that , preds = x .Something like that, I guess is what they would do. Anyway. There's a few different ways to do it. Okay. So hopefully you can see that this is really making it nice and easy for us to do our forward pass and our backward paths. Without all of this manual fiddling around. Every class now can be totally separately considered and can be combined. However want. We could create layers so you could try creating a bigger neural net if you want to, but we can refactor it more. So basically as a rule of thumb, when you see repeated like self.inp = inp.... That's a sign can refactor things. And so what we can do is a simple refactoring this to create a new class called module and modules gonna do those things. You said it's going to store the inputs and it's going to call something called self.forward in order to create our self.out because remember, that was one of the things we had again and again and again, self.out,  return it. And so now that's going to be a thing called forward which actually in this it doesn't do anything because the whole purpose of this module is to be inherited. When we call backward, it's going to call self.backward passing in self.out because notice all of our backwards always wanted to get hold of self.out  because we need it for the chain role. So let's pass that in and pass in those arguments that we start earlier.

## Refactor model

### Layers as classes

In [32]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [33]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b

    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [34]:
class Mse():
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [35]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [36]:
model = Model(w1, b1, w2, b2)

In [37]:
loss = model(x_train, y_train)

In [38]:
model.backward()

In [39]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Module.forward()

And so *args means take all of the arguments regardless whether it's zero one, two or more, and put them into a list. And then that's what happens when it's inside the actual signature. And then when you call a function using star, it says take this list and expand them into separate calling backward with each one separately. So now for relu, you look how much simpler it is. Let's copy the old relu and the new relu. So the old relu you had to do all this storing stuff manually and it had a little      self.stuff as well. But now we can get rid of all of that and just implement forward because that's the thing that's been called and that's the thing that we need to implement. And so now the forward is relu just as the one thing we want, which also makes the code much cleaner and more understandable. Ditto for backward. It just does. The one thing we want. So that's nice. Now we still have to multiply it by two after the chain rule manually, but same thing for linear, same thing for MSE. So these all look a lot nicer. And one thing to point out here is that there's often opportunities to manually speed things up when you create custom order grad functions in PyTorch. And here's an example. Look, this calculation is being done twice. It seems like a waste, doesn't it so at the cost of some memory, we could instead store that calculation as diff. Right. And I guess we'd have to store it fur use it later. So I don't need to be self.diff if and the cost of that memory, we could now remove this redundant calculation because we've done it once before already and stored it and just use it directly. And this is something that you can often do in neural nets. So there's this compromise between storing things, the memory use of that and then the computational speed up of not having to recalculate it. This is something we come across a lot. And so now we can call it in the same way, create a model passing in all of those layers. So you can see with our model, we're just so the model hasn't changed at this point. The definition was up here. We just passed in The weights for the layers calculate the loss called backward and look. It's the same right? Okay. So thankfully, PyTorch has written all this for us. And remember, according to rules of our game, once we've reimplemented it, we're allowed to use PyTorch as version. So PyTorch calls their version and nn.Module. And so it's exactly the same you inherit from an nn.Module. So if we want to create a linear layer just like this one, rather than inheriting our module, we will inherit from that module, but everything's exactly the same. So we create our, we can create our random numbers. So in this case, rather than passing in the already randomized weights, we're actually going to generate the random weights ourselves and the zeroed biases and. Then here's our linear layer, which you could also use Lin for that.

In [40]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [41]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [42]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [43]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [44]:
model = Model(w1, b1, w2, b2)

In [45]:
loss = model(x_train, y_train)

In [46]:
model.backward()

In [47]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

And of course, 

so define our forward. And why don't we need to define backward? Because PyTorch already knows the derivatives of all of the functions in PyTorch and it knows how to use the chain rule so we don't have to do backward at all. It will actually do that entirely for us, which is very cool. So we only need forward. We don't need backward.

 So let's create a model that uses that nn.Moddule. Otherwise it's exactly the same as before. And now we're going to use PyTorch as MSE loss because we've already implemented ourselves. It's very common to use torch.nn.functional as capital P. This is where lots of these handy functions live, including MSE loss. And so now you know why we need the colon comma None because you saw the problem if we don't have it. And so the model  call backward. And remember, we stored our gradients in something called (.g) PyTorch stores them in something called (.grad), but it's doing exactly the same thing. So there is the exact same values. All right. So let's see if there's any questions. Not yet. Okay. All right. If anybody in the class has any questions or comments about any of this, let me know. Remember to upvote questions that you're interested in.
 
 So we've we've created a matrix multiplication from scratch. 
 
 We've created linear layers, 
 
 we've created a complete backprop system of modules. 
 
 We can now calculate both the forward pass and the backward pass for linear layers and relus so we can create a multi-layer perceptron.
 
So we're now up to a point where we can train a model.

So let's do that mini batch training Notebook number 4. 



### Autograd

In [48]:
from torch import nn
import torch.nn.functional as F

In [49]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [50]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [51]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [52]:
l0 = model.layers[0]
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])